In [1]:
# Install required packages
!pip install -q transformers
# !pip install sentencepiece
# !pip install -q hazm
# !pip install parsivar
# !pip install -q clean-text[gpl]
# !pip install symspellpy
# !pip install keras_self_attention

In [2]:
# Import required packages
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from sklearn.utils import shuffle
# import hazm
# from cleantext import clean
# import plotly.express as px
# import plotly.graph_objects as go

# import os
# import re
# import json
# import copy
# import collections
# import sklearn.metrics as skm
# import sentencepiece
from transformers import BertConfig, BertTokenizer
from transformers import TFBertModel, TFBertForSequenceClassification
from transformers import glue_convert_examples_to_features

from keras.layers import GlobalMaxPooling1D
from keras.layers import *
from keras.models import Model
# from transformers import *
from transformers import TFBertModel, TFBertForSequenceClassification, BertModel
from transformers.modeling_tf_utils import get_initializer
# from keras_self_attention import SeqSelfAttention
from sklearn.model_selection import train_test_split

In [16]:
from tqdm.notebook import tqdm

In [20]:
import tensorflow as tf

### Load the data using Pandas

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import time
t1 = time.perf_counter()

In [5]:
# from hazm import word_tokenize
# import string

# def preproccess(text):
#   text = ''.join([i for i in text if not i.isdigit()]);
#   text = text.replace("#", "")
#   text = text.replace("ـ", "")
#   text = text.replace("؛", "")
#   text = text.replace("_", " ")
#   text = text.replace("،", " ")
#   text = text.replace("…", " ")
#   text = text.replace("«", " ")
#   text = text.replace("»", " ")
#   text = text.replace("٭", " ")
#   translator = str.maketrans(string.punctuation, ' '*len(string.punctuation))
#   text = text.translate(translator)
#   result = []
#   for token in word_tokenize(text):
#     result.append(token)
#   text = " ".join(result)
#   return text

In [6]:
emotion_dataset_train = pd.read_excel('drive/MyDrive/ShortPersianEmo 2023 Code/Data/ShortPersianEmo/train_fa.xlsx', header=None).dropna()
x_train = emotion_dataset_train[0]
y_train = emotion_dataset_train[1]

emotion_dataset_test = pd.read_excel('drive/MyDrive/ShortPersianEmo 2023 Code/Data/ShortPersianEmo/test_fa.xlsx', header=None).dropna()
x_test = emotion_dataset_test[0]
y_test = emotion_dataset_test[1]

In [7]:
from collections import Counter
cnt = Counter(y_train)
cnt = dict(cnt)
print(cnt)

{'SAD': 845, 'OTHER': 1263, 'HAPPY': 1462, 'ANGRY': 1012, 'FEAR': 342}


### Normalization / Preprocessing

In [8]:
data_train = emotion_dataset_train[[0, 1]]
data_train.columns = ['comment', 'label']

data_test = emotion_dataset_test[[0, 1]]
data_test.columns = ['comment', 'label']

## Train,Validation,Test split

In [9]:
test = data_test
train = data_train

x_train, y_train = train['comment'].values.tolist(), train['label'].values.tolist()
x_test, y_test = test['comment'].values.tolist(), test['label'].values.tolist()

print(train.shape)
print(test.shape)

(4924, 2)
(548, 2)


## TensorFlow

In [11]:
# general config
MAX_LEN = 100
TRAIN_BATCH_SIZE = 32
VALID_BATCH_SIZE = 32
TEST_BATCH_SIZE = 32

EPOCHS = 3
EEVERY_EPOCH = 1000
LEARNING_RATE = 2e-5
CLIP = 0.0

MODEL_NAME_OR_PATH = 'HooshvareLab/bert-base-parsbert-uncased'
OUTPUT_PATH = '/content/Arman_DS/tensorflow_model.bin'

# os.makedirs(os.path.dirname(OUTPUT_PATH), exist_ok=True)

In [12]:
labels = list(sorted(train['label'].unique()))
train['label_id'] = train['label'].apply(lambda t: labels.index(t))
test['label_id'] = test['label'].apply(lambda t: labels.index(t))
label2id = {label: i for i, label in enumerate(labels)}
id2label = {v: k for k, v in label2id.items()}

In [13]:
tokenizer = BertTokenizer.from_pretrained(MODEL_NAME_OR_PATH)
config = BertConfig.from_pretrained(MODEL_NAME_OR_PATH, **{ 'label2id': label2id, 'id2label': id2label})

vocab.txt:   0%|          | 0.00/1.22M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/434 [00:00<?, ?B/s]

### Input Embeddings / Dataset

In [14]:
class InputExample:
    """ A single example for simple sequence classification. """
    def __init__(self, guid, text_a, text_b=None, label=None):
        """ Constructs a InputExample. """
        self.guid = guid
        self.text_a = text_a
        self.text_b = text_b
        self.label = label
def make_examples(tokenizer, x, y=None, maxlen=MAX_LEN, output_mode="classification", is_tf_dataset=True):
    examples = []
    y = y if isinstance(y, list) or isinstance(y, np.ndarray) else [None] * len(x)
    for i, (_x, _y) in tqdm(enumerate(zip(x, y)), position=0, total=len(x)):
        guid = "%s" % i
        label = int(_y)
        if isinstance(_x, str):
            text_a = _x
            text_b = None
        else:
            assert len(_x) == 2
            text_a = _x[0]
            text_b = _x[1]
        examples.append(InputExample(guid=guid, text_a=text_a, text_b=text_b, label=label))
    features = glue_convert_examples_to_features(
        examples,
        tokenizer,
        maxlen,
        output_mode=output_mode,
        label_list=list(np.unique(y)))
    all_input_ids = []
    all_attention_masks = []
    all_token_type_ids = []
    all_labels = []
    for f in tqdm(features, position=0, total=len(examples)):
        if is_tf_dataset:
            all_input_ids.append(tf.constant(f.input_ids))
            all_attention_masks.append(tf.constant(f.attention_mask))
            all_token_type_ids.append(tf.constant(f.token_type_ids))
            all_labels.append(tf.constant(f.label))
        else:
            all_input_ids.append(f.input_ids)
            all_attention_masks.append(f.attention_mask)
            all_token_type_ids.append(f.token_type_ids)
            all_labels.append(f.label)
    if is_tf_dataset:
        dataset = tf.data.Dataset.from_tensor_slices(({
            'input_ids': all_input_ids,
            'attention_mask': all_attention_masks,
            'token_type_ids': all_token_type_ids
        }, all_labels))

        return dataset, features
    xdata = [np.array(all_input_ids), np.array(all_attention_masks), np.array(all_token_type_ids)]
    ydata = all_labels
    return [xdata, ydata], features

In [21]:
train_dataset_base, train_examples = make_examples(tokenizer, x_train, np.asarray(train['label_id']), maxlen=MAX_LEN)

test_dataset_base, test_examples = make_examples(tokenizer, x_test, np.asarray(test['label_id']), maxlen=MAX_LEN)
[xtest, ytest], test_examples = make_examples(tokenizer, x_test, np.asarray(test['label_id']), maxlen=MAX_LEN, is_tf_dataset=False)

  0%|          | 0/4924 [00:00<?, ?it/s]

  0%|          | 0/4924 [00:00<?, ?it/s]

  0%|          | 0/548 [00:00<?, ?it/s]

  0%|          | 0/548 [00:00<?, ?it/s]

  0%|          | 0/548 [00:00<?, ?it/s]

  0%|          | 0/548 [00:00<?, ?it/s]

In [22]:
def get_training_dataset(dataset, batch_size):
    dataset = dataset.repeat()
    dataset = dataset.shuffle(2048)
    dataset = dataset.batch(batch_size)
    return dataset

def get_validation_dataset(dataset, batch_size):
    dataset = dataset.batch(batch_size)
    return dataset

In [23]:
train_dataset = get_training_dataset(train_dataset_base, TRAIN_BATCH_SIZE)
test_dataset = get_validation_dataset(test_dataset_base, VALID_BATCH_SIZE)

train_steps = len(train_examples) // TRAIN_BATCH_SIZE
test_steps = len(test_examples) // VALID_BATCH_SIZE

### Model

In [24]:
import math
def ceiltoup(x):
  return math.ceil(x * 100) / 100.0

f1_macro = []
accuracy = []
input_ids = tf.keras.layers.Input(shape=(MAX_LEN,), dtype=tf.int32, name="input_ids")
attention_masks = tf.keras.layers.Input(shape=(MAX_LEN,), dtype=tf.int32, name="attention_mask")
token_type_ids = tf.keras.layers.Input(shape=(MAX_LEN,), dtype=tf.int32, name="token_type_ids")
model = TFBertModel.from_pretrained(MODEL_NAME_OR_PATH, config=config, from_pt=True)
model.trainable = True
output = model(input_ids, attention_mask=attention_masks, token_type_ids=token_type_ids)
output = Dense(len(labels), activation='softmax', name='dense_out')(output['pooler_output'])
model = Model(inputs=[input_ids, attention_masks, token_type_ids], outputs=output)
optimizer = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE)
loss = tf.keras.losses.SparseCategoricalCrossentropy()
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
model.compile(optimizer=optimizer, loss=loss, metrics=[metric])
EarlyStopping_callback = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', min_delta=1e-3, patience=2, verbose=1, mode='auto', restore_best_weights=True)
r = model.fit( train_dataset, validation_data=test_dataset, steps_per_epoch=train_steps, validation_steps=test_steps, epochs=20, batch_size=32, verbose=1, callbacks=[EarlyStopping_callback])
predictions = model.predict(xtest)
ypred = predictions.argmax(axis=-1).tolist()

pytorch_model.bin:   0%|          | 0.00/654M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint

Epoch 1/20
153/153 [==============================] - 145s 666ms/step - loss: 0.9622 - accuracy: 0.6191 - val_loss: 0.7983 - val_accuracy: 0.6838
Epoch 2/20
153/153 [==============================] - 101s 660ms/step - loss: 0.6181 - accuracy: 0.7684 - val_loss: 0.8053 - val_accuracy: 0.7114
Epoch 3/20
153/153 [==============================] - 99s 647ms/step - loss: 0.3713 - accuracy: 0.8664 - val_loss: 0.9069 - val_accuracy: 0.7096
Epoch 4/20
153/153 [==============================] - 101s 659ms/step - loss: 0.1937 - accuracy: 0.9377 - val_loss: 1.0501 - val_accuracy: 0.7096
Epoch 4: early stopping
18/18 [==============================] - 7s 213ms/step


In [25]:
t2 = time.perf_counter()
print('time taken to run:',t2-t1)

time taken to run: 908.6513209799999


In [30]:
len(y_test)

548

In [29]:
len(y_train)

4924